In [1]:
import pandas as pd
import numpy as np
import scipy.sparse

from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import json

from datetime import datetime

import lightgbm as lgb
import gc

from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.test.utils import datapath

import re
from nltk.corpus import stopwords

from razdel import tokenize

import pymorphy2

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

/home/tamara/.local/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


##### Импортируем датасет с новостями и данными пользователей

In [2]:
news = pd.read_csv('articles.csv')
news.head(3)

,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [3]:
users = pd.read_csv('users_articles.csv')
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


#### Получаем векторное представление новостей

##### Добавим словарь со стоп словами

In [4]:
stopword_ru = stopwords.words('russian')

morph = pymorphy2.MorphAnalyzer()

len(stopword_ru)

151

In [5]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

##### Очистка текста

In [6]:
# Функция для очистки текста

def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

# Функция для лемматизации текста

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

##### Запустим очистку текста

In [7]:
%%time
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

CPU times: user 1min 42s, sys: 3.08 s, total: 1min 45s
Wall time: 2min


##### Запустим лемматизацию текста

In [8]:
%%time
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 12min 2s, sys: 2.12 s, total: 12min 4s
Wall time: 13min 18s


##### Обучим нашу модель.
##### Сформируем список текстов, разбив пробелами.

In [9]:
# словарь текстов
texts = [t for t in news['title'].values]

# Создадим словарь и вектор
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [10]:
%time
# Тренируем модель на векторе
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 21.9 µs


In [46]:
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

##### Функция, которая возвращает векторное представление новости

In [47]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

##### Составим матрицу вероятности попадания слов в статью

In [48]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.0,0.868966,0.00000,0.0,0.048010,0.0,0.000000,0.074615,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
1,4896,0.0,0.0,0.000000,0.00000,0.0,0.243632,0.0,0.344851,0.000000,...,0.0,0.0,0.297644,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
2,4897,0.0,0.0,0.000000,0.00000,0.0,0.220460,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.282352,0.0,0.0,0.068297
3,4898,0.0,0.0,0.000000,0.59719,0.0,0.101513,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.290255,0.0,0.0,0.000000
4,4899,0.0,0.0,0.000000,0.00000,0.0,0.146372,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.826528,0.0,0.000000,0.0,0.0,0.000000


#### Получаем векторное представление пользователей

In [49]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

##### Реализуем функцию формирования вектора пользователя. Модифицируем ее, чтобы можно было задавать значение по среднему, медиане или максимуму

In [50]:
user_articles_list = users['articles'].iloc[33]

In [51]:
def get_user_embedding_mean(user_articles_list):
    user_articles_list_ = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list_])
    user_vector = np.mean(user_vector, 0)
    return user_vector


def get_user_embedding_median(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector


def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

##### Получим вектора для 

In [52]:
get_user_embedding_mean(user_articles_list)

array([0.        , 0.01373544, 0.13953323, 0.04370556, 0.0491291 ,
       0.        , 0.0047903 , 0.        , 0.01928422, 0.        ,
       0.01935515, 0.        , 0.02940983, 0.00282792, 0.26815222,
       0.00229112, 0.06307563, 0.09556313, 0.03167733, 0.05131089,
       0.        , 0.        , 0.        , 0.08712595, 0.0636726 ])

In [53]:
get_user_embedding_median(user_articles_list)

array([0.        , 0.        , 0.11212976, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.30560724,
       0.        , 0.00941558, 0.04080943, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.05610815, 0.        ])

In [54]:
get_user_embedding_max(user_articles_list)

array([0.        , 0.04764197, 0.41012716, 0.26223335, 0.21201667,
       0.        , 0.02874179, 0.        , 0.0580928 , 0.        ,
       0.1161309 , 0.        , 0.10741234, 0.01696753, 0.48292279,
       0.01374674, 0.32887152, 0.31679794, 0.17538516, 0.18895453,
       0.        , 0.        , 0.        , 0.25841689, 0.37132806])

##### Свернем получение эмбедингов по разным критериям в одну функцию

In [55]:
def get_embegings(user_vector):
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: user_vector(x), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
    
    target = pd.read_csv("users_churn.csv")
    
    X = pd.merge(user_embeddings, target, 'left')
    
    return X

##### Свернем предсказание в функцию

In [56]:
def model_prediction(X):
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)
    
    logreg = LogisticRegression()
    #обучим 
    logreg.fit(X_train, y_train)

    #наши прогнозы для тестовой выборки
    preds = logreg.predict_proba(X_test)[:, 1]
    
    return y_test, preds

##### Произведем расчеты

In [57]:
X_mean = get_embegings(get_user_embedding_mean)
X_median = get_embegings(get_user_embedding_median)
X_max = get_embegings(get_user_embedding_max)

In [58]:
y_mean_train, y_mean_test = model_prediction(X_mean)
y_median_train, y_median_test = model_prediction(X_median)
y_max_train, y_max_test = model_prediction(X_max)

#### Рассчитаесм метрики Precision, Recall, F_score, Roc_Auc

In [74]:
def calc_metrics(y_test, preds):
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    return {'Best Threshold': thresholds[ix], 'F-Score': fscore[ix], 'Precision': precision[ix], 'Recall': recall[ix]}

In [75]:
result_mean = calc_metrics(y_mean_train, y_mean_test)
result_median = calc_metrics(y_median_train, y_median_test)
result_max = calc_metrics(y_max_train, y_max_test)

In [76]:
results = pd.DataFrame({
    'Mean': pd.Series(result_mean),
    'Median': pd.Series(result_median),
    'Max': pd.Series(result_max)
})
results

,Mean,Median,Max
Best Threshold,0.234820,0.327184,0.424411
F-Score,0.660000,0.745263,0.828633
Precision,0.557746,0.769565,0.884259
Recall,0.808163,0.722449,0.779592
